In [1]:
import pandas as pd
import numpy as np
# from pandas.io.json import json_normalize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
import string
# from nltk.stem.porter import *
import nltk
from nltk.corpus import stopwords
# from nltk.corpus import wordnet
from ast import literal_eval
from tqdm import tqdm

In [ ]:
# nltk.download("stopwords")

# Content-based Filtering

## model 1: TF-IDF for movie `overview`,  `tagline`, and `keywords`

### Natual Language Processing

In [3]:
PUNCTUATION = string.punctuation
# stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")
STOPWORDS = stopwords.words("english")

In [2]:
def nlp(x):
    """
    Natural Language Processing
    
    Args:
        x (str)
    
    Returns:
        (str)
    """
    x = x.lower() # lowercase string
    x = [c for c in x if not (c in PUNCTUATION)] # non-punct characters
    x = "".join(x) # convert back to string
    words = x.strip().split() # tokenizes
    words = [w for w in words if w not in STOPWORDS] # remove stopwords
    return " ".join(stemmer.stem(w) for w in words)

In [4]:
# movie_meta = pd.read_csv("the-movies-dataset/movies_metadata.csv", dtype=str).drop_duplicates()
movie_meta = pd.read_parquet("the-movies-dataset/processed_data.parquet", engine="fastparquet").reset_index(drop=True)
# movie_meta.id = movie_meta.id.astype(int)
# movie_meta.overview.fillna("", inplace=True)
# movie_meta.tagline.fillna("", inplace=True)
movie_meta

,id,title,cast,keywords,genres,vote_average,vote_count,overview,release_date,popularity,tagline,director
0,862,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",7.7,5415,"Led by Woody, Andy's toys live happily in his ...",23950,21.946943,,[John Lasseter]
1,8844,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, new home, recluse,...","[Adventure, Fantasy, Family]",6.9,2413,When siblings Judy and Peter discover an encha...,23952,17.015539,Roll the dice and unleash the excitement!,[Joe Johnston]
2,15602,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Da...","[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]",6.5,92,A family wedding reignites the ancient feud be...,23952,11.7129,Still Yelling. Still Fighting. Still Ready for...,[Howard Deutch]
3,31357,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]",6.1,34,"Cheated on, mistreated and stepped on, the wom...",23952,3.859495,Friends are the people who let you be yourself...,[Forest Whitaker]
4,11862,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...",[Comedy],5.7,173,Just when George Banks has recovered from his ...,23942,8.387519,Just When His World Is Back To Normal... He's ...,[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...
45364,30840,Robin Hood,"[Patrick Bergin, Uma Thurman, David Morrissey,...",[],"[Drama, Action, Romance]",5.7,26,"Yet another version of the classic epic, with ...",23897,5.683753,,[John Irvin]
45365,111109,Century of Birthing,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[artist, play, pinoy]",[Drama],9.0,3,An artist struggles to finish his work while a...,24143,0.178241,,[Lav Diaz]
45366,67758,Betrayal,"[Erika Eleniak, Adam Baldwin, James Remar, Dam...",[],"[Action, Drama, Thriller]",3.8,6,"When one of her hits goes wrong, a professiona...",24044,0.903007,A deadly game of wits.,[]
45367,227506,Satan Triumphant,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[],[],0.0,0,"In a small town live two brothers, one a minis...",23014,0.003503,,[Yakov Protazanov]


In [ ]:
%%time
movie_meta["description"] = movie_meta.overview + movie_meta.tagline# + keywords.extract.astype(str)
movie_meta["description_nlp"] = movie_meta["description"].apply(nlp)

In [ ]:
movie_meta.iloc[1]["description"]

In [ ]:
movie_meta.iloc[1]["description_nlp"]

### TF-IDF matrix

In [ ]:
tfidf = TfidfVectorizer(analyzer="word", stop_words="english")

In [ ]:
tfidf_matrix = tfidf.fit_transform(movie_meta.overview_nlp)

In [ ]:
tfidf_matrix.shape

### cosine similarity matrix

In [ ]:
movie_desc_sim = linear_kernel(tfidf_matrix)#, tfidf_matrix)

In [ ]:
movie_desc_sim.shape

### index mapping

In [ ]:
# movie_id = movie_meta.id
TITLE_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.title)
ID_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.id)
INDEX_TO_TITLE = pd.Series(movie_meta.title, index=movie_meta.index)
# ID_TO_INDEX = pd.Series(movie_meta.index, index=movie_meta.id)

In [ ]:
TITLE_TO_INDEX

In [ ]:
TITLE_TO_INDEX["Jumanji"]

In [ ]:
ID_TO_INDEX

In [ ]:
ID_TO_INDEX["862"]

## movie description cosine similarity model

In [ ]:
def get_recommendations(movie_id, cosine_sim, title_series):
    idx = ID_TO_INDEX[str(movie_id)]
    print(f"Recommend for `{INDEX_TO_TITLE[idx]}`")
    # TODO: duplicate name
#     if len(idx) > 1:
#         print(type(idx))
#         idx = idx.iloc[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return title_series.iloc[movie_indices]

In [ ]:
get_recommendations("238", movie_desc_sim, movie_meta.title)

In [ ]:
TITLE_TO_INDEX["The Godfather"]

In [ ]:
movie_meta[movie_meta.title=="The Godfather"]

## one-hot similarity model

In [ ]:
def one_hot_encoding(list_, db):
    res = np.zeros(len(db), dtype=int)
    for item in list_:
        idx_list = np.where(db == item)[0]
        if len(idx_list) == 0: continue
        res[idx_list[0]] = 1
    return list(res)#.reshape(1, -1)

In [ ]:
# def one_hot_encoding(list_, db):
#     res = np.zeros_like(db)
#     for item in list_:
#         res[np.where(db == item)[0][0]] = 1
#     return res.reshape(1, -1)

In [ ]:
def get_count(pd_series, threshold):
    """
    Get item count from list of itmes of the column.
    
    Args:
        pd_series (pd.Series)
        threshold (int)
    
    Returns:
        (pd.Series)
        (list)
    """
    cnt = pd_series.explode().to_frame("item").reset_index(drop=True).groupby("item").size().sort_values(ascending=False)
    series = cnt[cnt >= threshold]
    return series, series.index.tolist()

In [ ]:
_, DIRECTORS = get_count(movie_meta.director, threshold=40)
DIRECTORS = np.array(DIRECTORS)#, dtype=str)
DIRECTORS

In [ ]:
_, KEYWORDS = get_count(movie_meta.keywords, threshold=500)
KEYWORDS = np.array(KEYWORDS)#, dtype=str)
KEYWORDS

In [ ]:
_, CASTS = get_count(movie_meta.cast, threshold=100)
CASTS = np.array(CASTS)#, dtype=str)
CASTS

In [ ]:
GENRES = movie_meta.genres.explode().unique()
GENRES

In [ ]:
one_hot_name = GENRES.tolist() + KEYWORDS.tolist() + CASTS.tolist() + DIRECTORS.tolist()
one_hot_name

In [ ]:
movie_meta["genres"].apply(one_hot_encoding, db=GENRES)

In [ ]:
movie_meta["keywords"].apply(one_hot_encoding, db=KEYWORDS)

In [ ]:
movie_meta["cast"].apply(one_hot_encoding, db=CASTS)

In [ ]:
movie_meta["director"].apply(one_hot_encoding, db=DIRECTORS)

In [ ]:
movie_meta["one_hot"] = movie_meta["genres"].apply(one_hot_encoding, db=GENRES) + \
                        movie_meta["keywords"].apply(one_hot_encoding, db=KEYWORDS) + \
                        movie_meta["cast"].apply(one_hot_encoding, db=CASTS) + \
                        movie_meta["director"].apply(one_hot_encoding, db=DIRECTORS)

In [ ]:
len(movie_meta.one_hot[0])

### Jaccard Similarity

In [ ]:
# from scipy.spatial.distance import jaccard, pdist, squareform
# from sklearn.metrics import pairwise_distances
# from sklearn.metrics import jaccard_score
# from nltk.metrics.distance import jaccard_distance

In [ ]:
# %%time
# JACCARD_SIM = 1 - pdist(pd.DataFrame(movie_meta['one_hot'].explode().values.tolist()), "jaccard")
# JACCARD_SIM

In [ ]:
# %%time
# JACCARD_SIM_MAT = pd.DataFrame(squareform(JACCARD_SIM), index=movie_meta.index, columns= movie_meta.index)

In [5]:
def Jaccard(s1, s2):
    assert isinstance(s1, set)
    assert isinstance(s2, set)
    nom = len(s1.intersection(s2))
    den = len(s1.union(s2))
    return nom / den

In [10]:
movie_meta["items_for_jaccard"] = movie_meta["cast"] + movie_meta["keywords"] + movie_meta["genres"] + movie_meta["director"]
movie_meta

,id,title,cast,keywords,genres,vote_average,vote_count,overview,release_date,popularity,tagline,director,items_for_jaccard
0,862,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",7.7,5415,"Led by Woody, Andy's toys live happily in his ...",23950,21.946943,,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,8844,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, new home, recluse,...","[Adventure, Fantasy, Family]",6.9,2413,When siblings Judy and Peter discover an encha...,23952,17.015539,Roll the dice and unleash the excitement!,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,15602,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Da...","[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]",6.5,92,A family wedding reignites the ancient feud be...,23952,11.7129,Still Yelling. Still Fighting. Still Ready for...,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Sophia Loren, Da..."
3,31357,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]",6.1,34,"Cheated on, mistreated and stepped on, the wom...",23952,3.859495,Friends are the people who let you be yourself...,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,11862,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...",[Comedy],5.7,173,Just when George Banks has recovered from his ...,23942,8.387519,Just When His World Is Back To Normal... He's ...,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45364,30840,Robin Hood,"[Patrick Bergin, Uma Thurman, David Morrissey,...",[],"[Drama, Action, Romance]",5.7,26,"Yet another version of the classic epic, with ...",23897,5.683753,,[John Irvin],"[Patrick Bergin, Uma Thurman, David Morrissey,..."
45365,111109,Century of Birthing,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[artist, play, pinoy]",[Drama],9.0,3,An artist struggles to finish his work while a...,24143,0.178241,,[Lav Diaz],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45366,67758,Betrayal,"[Erika Eleniak, Adam Baldwin, James Remar, Dam...",[],"[Action, Drama, Thriller]",3.8,6,"When one of her hits goes wrong, a professiona...",24044,0.903007,A deadly game of wits.,[],"[Erika Eleniak, Adam Baldwin, James Remar, Dam..."
45367,227506,Satan Triumphant,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[],[],0.0,0,"In a small town live two brothers, one a minis...",23014,0.003503,,[Yakov Protazanov],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [7]:
def remove_empty_list(x):
#     print(type(x))
    assert isinstance(x, list)
    if len(x) == 0:
        return None
    else:
        return x

In [ ]:
# movie_meta.items_for_jaccard.apply(len).sort_values().head(45369-45116+1)

In [11]:
# movie_meta["items_for_jaccard"].dropna(inplace=True)
movie_meta["items_for_jaccard"] = movie_meta["items_for_jaccard"].apply(remove_empty_list).dropna()


In [12]:
movie_meta_new = movie_meta.dropna().reset_index()
movie_meta_new

,index,id,title,cast,keywords,genres,vote_average,vote_count,overview,release_date,popularity,tagline,director,items_for_jaccard
0,0,862,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[jealousy, toy, boy, friendship, friends, riva...","[Animation, Comedy, Family]",7.7,5415,"Led by Woody, Andy's toys live happily in his ...",23950,21.946943,,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,1,8844,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[board game, disappearance, new home, recluse,...","[Adventure, Fantasy, Family]",6.9,2413,When siblings Judy and Peter discover an encha...,23952,17.015539,Roll the dice and unleash the excitement!,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,2,15602,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Sophia Loren, Da...","[fishing, best friend, duringcreditsstinger, o...","[Romance, Comedy]",6.5,92,A family wedding reignites the ancient feud be...,23952,11.7129,Still Yelling. Still Fighting. Still Ready for...,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Sophia Loren, Da..."
3,3,31357,Waiting to Exhale,"[Whitney Houston, Angela Bassett, Loretta Devi...","[based on novel, interracial relationship, sin...","[Comedy, Drama, Romance]",6.1,34,"Cheated on, mistreated and stepped on, the wom...",23952,3.859495,Friends are the people who let you be yourself...,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,4,11862,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[baby, midlife crisis, confidence, aging, daug...",[Comedy],5.7,173,Just when George Banks has recovered from his ...,23942,8.387519,Just When His World Is Back To Normal... He's ...,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45111,45364,30840,Robin Hood,"[Patrick Bergin, Uma Thurman, David Morrissey,...",[],"[Drama, Action, Romance]",5.7,26,"Yet another version of the classic epic, with ...",23897,5.683753,,[John Irvin],"[Patrick Bergin, Uma Thurman, David Morrissey,..."
45112,45365,111109,Century of Birthing,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[artist, play, pinoy]",[Drama],9.0,3,An artist struggles to finish his work while a...,24143,0.178241,,[Lav Diaz],"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe..."
45113,45366,67758,Betrayal,"[Erika Eleniak, Adam Baldwin, James Remar, Dam...",[],"[Action, Drama, Thriller]",3.8,6,"When one of her hits goes wrong, a professiona...",24044,0.903007,A deadly game of wits.,[],"[Erika Eleniak, Adam Baldwin, James Remar, Dam..."
45114,45367,227506,Satan Triumphant,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[],[],0.0,0,"In a small town live two brothers, one a minis...",23014,0.003503,,[Yakov Protazanov],"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa..."


In [55]:
np.save("jaccard_sim_idx_mapping", movie_meta_new["index"].values)

In [13]:
%%time
JACCARD_SIM = np.zeros((len(movie_meta_new.index), len(movie_meta_new.index)))
JACCARD_SIM, JACCARD_SIM.shape

CPU times: user 249 µs, sys: 169 µs, total: 418 µs
Wall time: 341 µs


(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), (45116, 45116))

In [14]:
%%time
t = tqdm()
t.clear()
for i1 in tqdm(range(len(movie_meta_new.index))):
    for i2 in range(i1 + 1, len(movie_meta_new.index)):
        s1 = set(movie_meta_new.items_for_jaccard[i1])
        s2 = set(movie_meta_new.items_for_jaccard[i2])
#         if not s1.difference(s2): continue
        try:
            sim = Jaccard(s1, s2)
        except:
            print(f"Jaccard has trouble: (s1, s2) = ({s1}, {s2})")
            sim = 0
        JACCARD_SIM[i1, i2] = sim
        JACCARD_SIM[i2, i1] = sim

                  
  0%|          | 142/45116 [04:19<22:23:54,  1.79s/it]


  1%|          | 283/45116 [08:38<21:22:11,  1.72s/it]


  1%|          | 424/45116 [12:53<23:19:27,  1.88s/it]


  1%|▏         | 565/45116 [17:00<21:20:45,  1.72s/it]


  2%|▏         | 706/45116 [21:02<21:07:34,  1.71s/it]


...

 97%|█████████▋| 43837/45116 [10:52:06<01:07, 18.81it/s]


 98%|█████████▊| 44220/45116 [10:52:23<00:33, 27.11it/s]


 99%|█████████▉| 44860/45116 [10:52:38<00:03, 84.80it/s]


100%|██████████| 45116/45116 [10:52:40<00:00,  1.15it/s] 

CPU times: user 10h 50min 50s, sys: 1min 53s, total: 10h 52min 43s
Wall time: 10h 52min 40s


In [18]:
np.savez_compressed("JACCARD_SIM.npz", JACCARD_SIM)

In [38]:
a = np.load("JACCARD_SIM.npz")

In [48]:
a.files

['arr_0']

In [47]:
%%time
aa = a["arr_0"]

CPU times: user 28.3 s, sys: 6.32 s, total: 34.6 s
Wall time: 36.5 s


In [49]:
aa[0]

array([0.        , 0.01818182, 0.02777778, ..., 0.        , 0.        ,
       0.        ])